In [1]:
!rm -r data
import os 

try:
  os.mkdir("./data")
except:
  pass

!wget https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt -P ./data

rm: cannot remove 'data': No such file or directory
--2021-05-13 12:49:21--  https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘./data/input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2021-05-13 12:49:22 (32.2 MB/s) - ‘./data/input.txt’ saved [1115394/1115394]



In [2]:
import torch
import torch.nn as nn

In [3]:
!pip install unidecode

     |████████████████████████████████| 245kB 10.2MB/s 


In [4]:
import unidecode
import string
import random
import re
import time, math

In [5]:
num_epochs = 2000
print_every = 100
plot_every = 10

# chunk에 대한 설명은 아래 함수정의하면서 하겠습니다.
chunk_len = 200

hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

In [6]:
# import 했던 string에서 출력가능한 문자들을 다 불러옵니다. 
all_characters = string.printable

# 출력가능한 문자들의 개수를 저장해놓습니다.
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


In [7]:
# 앞서 다운받은 텍스트 파일을 열어줍니다.

file = unidecode.unidecode(open('./data/input.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 1115394


In [8]:
# 이 함수는 텍스트 파일의 일부분을 랜덤하게 불러오는 코드입니다.
def random_chunk():
    # (시작지점 < 텍스트파일 전체길이 - 불러오는 텍스트의 길이)가 되도록 시작점과 끝점을 정합니다.
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

his punishment was cruel death.
Who sued to me for him? who, in my rage,
Kneel'd at my feet, and bade me be advised
Who spake of brotherhood? who spake of love?
Who told me how the poor soul did forsak


In [9]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


In [10]:
# 랜덤한 텍스트 chunk를 불러와서 이를 입력과 목표값을 바꿔주는 함수입니다.
# 예를 들어 pytorch라는 문자열이 들어오면 입력은 pytorc / 목표값은 ytorch 가 됩니다.
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [17]:
class GNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(GNN, self).__init__()
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        # 임베딩 함수에 대한 설명은 책과 공식 도큐먼트를 참고하시길 바랍니다.
        # https://pytorch.org/docs/stable/nn.html?highlight=embedding#torch.nn.Embedding
        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.gnn = nn.GRU(self.embedding_size,self.hidden_size,self.num_layers)
        self.decoder = nn.Linear(self.hidden_size, self.output_size)
        
    
    def forward(self, input, hidden):
        out = self.encoder(input.view(1,-1))
        out,hidden = self.gnn(out,hidden)
        out = self.decoder(out.view(batch_size,-1))
        return out,hidden

    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden
    
model = GNN(n_characters, embedding_size, hidden_size, n_characters, num_layers)

In [18]:
model = GNN(input_size=n_characters, 
            embedding_size=embedding_size,
            hidden_size=hidden_size, 
            output_size=n_characters, 
            num_layers=2)

In [19]:
# 모델 테스트 

inp = char_tensor("A")
print(inp)
hidden = model.init_hidden()
print(hidden.size())
out,hidden = model(inp,hidden)
print(out.size())

tensor([36])
torch.Size([2, 1, 100])
torch.Size([1, 100])


In [20]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [21]:
# 임의의 문자(start_str)로 시작하는 길이 200짜리 모방 글을 생성하는 코드입니다.
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden = model(x,hidden)

        # 여기서 max값을 사용하지 않고 multinomial을 사용하는 이유는 만약 max 값만 쓰는 경우에
        # 생성되는 텍스트가 다 the the the the the 이런식으로 나오기 때문입니다.
        # multinomial 함수를 통해 높은 값을 가지는 문자들중에 램덤하게 다음 글자를 뽑아내는 방식으로 자연스러운 텍스트를 생성해냅니다.
        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

In [22]:
for i in range(num_epochs):
    # 랜덤한 텍스트 덩어리를 샘플링하고 이를 인덱스 텐서로 변환합니다. 
    inp,label = random_training_set()
    hidden = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden = model(x,hidden)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n","="*100)


 tensor([4.6251], grad_fn=<DivBackward0>) 

b!BAJ0sw=xCql\mR
f/>W+0|DKNzkp!`	0l$I{I()z0!:^&W"YYZxJdo.gc5;q\2([L/_znp
G![E?ar"O"L:--ud)zUU!/b{.|ktc_r`hT<?`]#u>EE2":=hjn#,{tQntzJrE7c!aO?91j)JT}Aaf)&N6wYvv(.sBww1hQ#De

 tensor([2.5909], grad_fn=<DivBackward0>) 

by ms mame rooir,
Hou tou My hel wo meno.-

Ai the.

Aan nigurer.

EBI:
Thous setiboe ithe.

Oat mil at aft for ane the alg oo risrlith cee belald I.

Eene bust dat bod inde anl, bstincenfd bee
Pfomill

 tensor([2.2281], grad_fn=<DivBackward0>) 

b)y ald hay als me, withe
The thid you sour the thofer
There danst; to Come my hing mrertee thes to norlour storur no esong bmet Dhangt bo hton,
Tor me are youf wor thoud cave frace seaniefe hart sif n

 tensor([2.0429], grad_fn=<DivBackward0>) 

bxionce Not teve of dirs of to the kjyour apt since our the moll luntd soup I I hather foreng the do; so no the bore to he al's of freow,
Bere he my feat jor thy not menm bove noke that my.

GUKRNS Lo 

 tensor([2.0626], grad_fn=<DivBackward

KeyboardInterrupt: ignored